# Multi agent customer support system

In [26]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [27]:
from crewai import Agent, Task, Crew

In [45]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key() 
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Role Playing, Focus and Cooperation

In [46]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
	verbose=True
)

- By not setting `allow_delegation=False`, `allow_delegation` takes its default value of being `True`.
- This means the agent _can_ delegate its work to another agent which is better suited to do a particular task. 

In [47]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
    #allow_delegation=False, cant delegate 
	verbose=True
)

* **Role Playing**: Both agents have been given a role, goal and backstory.
* **Focus**: Both agents have been prompted to get into the character of the roles they are playing.
* **Cooperation**: Support Quality Assurance Agent can delegate work back to the Support Agent, allowing for these agents to work together.

## Tools, Guardrails and Memory

### Tools

In [48]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool

### Possible Custom Tools
- Load customer data
- Tap into previous conversations
- Load data from a CRM
- Checking existing bug reports
- Checking existing feature requests
- Checking ongoing tickets
- ... and more

In [49]:
search_tool = SerperDevTool()

In [50]:
scrape_tool = ScrapeWebsiteTool()

In [52]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
    #website_url="https://niplan-market.onrender.com/"
)

##### Different Ways to Give Agents Tools

- Agent Level: The Agent can use the Tool(s) on any Task it performs.
- Task Level: The Agent will only use the Tool(s) when performing that specific Task.

**Note**: Task Tools override the Agent Tools.

### Creating Tasks
- You are passing the Tool on the Task Level.

In [53]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

In [54]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)


### Creating the Crew

#### Memory
- Setting `memory=True` when putting the crew together enables Memory.

In [55]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=True,
  memory=True
)

## Running the Crew

Guardrails
By running the execution below, you can see that the agents and the responses are within the scope of what we expect from them.

In [56]:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: e94d5468-9c5c-4380-b6fd-7914bd4898ee                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────── 🧠 Retrieved Memory ──────────────────────────────────────────────╮
│                                                                                                                 │
│  Historical Data:                                                                                               │
│  - Include more specific examples to illustrate the points made in the response.                                │
│  - Provide additional links or resources for further reading on the topic of adding memory to a Crew in         │
│  CrewAI.                                                                                                        │
│  - Consider using a more interactive format such as diagrams or visual aids to enhance understanding.           │
│  Recent Insights:                                                                                               │
│  - I now can give a great answer                                                                                │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│                                                                                                                 │
│  Hello DeepLearningAI Team,                                                                                     │
│                                                                                                                 │
│  Thank you for reaching out to us regarding how to add memory capabilities to a Crew in CrewAI....              │
│                                                                                                                 │
╰─────────────────────────────────────────── Retrieval Time: 4129.47ms ───────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Support Representative                                                                           │
│                                                                                                                 │
│  Task: DeepLearningAI just reached out with a super important ask:                                              │
│  I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you   │
│  provide guidance?                                                                                              │
│                                                                                                                 │
│  Andrew Ng from DeepLearningAI is the one that reached out. Make sure to use everything you know to provide     │
│  the best support possible.You must strive to provide a complete and accurate response to the customer's        │
│  inquiry.                                                                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Support Representative                                                                           │
│                                                                                                                 │
│  Thought: I need to gather additional information on how to add memory to a Crew in CrewAI specifically. I      │
│  should utilize the available tools to gather insights and provide a detailed and informative response to       │
│  DeepLearningAI's inquiry.                                                                                      │
│                                                                                                                 │
│  Using Tool: Read website content                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{}"                                                                                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────── 🧠 Retrieved Memory ──────────────────────────────────────────────╮
│                                                                                                                 │
│  Historical Data:                                                                                               │
│  - Provide more specific feedback related to the content of the response                                        │
│  - Ensure the response directly addresses all points of the customer's inquiry                                  │
│  - Include examples or case studies to further illustrate key points                                            │
│  - Include more specific details and examples to support the recommendations provided.                          │
│  - Ensure a smoother flow between the sections for better readability and understanding.                        │
│  - Consider incorporating visual aids or illustrations to enhance the explanation of comp...                    │
│                                                                                                                 │
╰─────────────────────────────────────────── Retrieval Time: 6692.82ms ───────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Support Quality Assurance Specialist                                                                    │
│                                                                                                                 │
│  Task: Review the response drafted by the Senior Support Representative for DeepLearningAI's inquiry. Ensure    │
│  that the answer is comprehensive, accurate, and adheres to the high-quality standards expected for customer    │
│  support.                                                                                                       │
│  Verify that all parts of the customer's inquiry have been addressed thoroughly, with a helpful and friendly    │
│  tone.                                                                                                          │
│  Check for references and sources used to  find the information, ensuring the response is well-supported and    │
│  leaves no questions unanswered.                                                                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Support Quality Assurance Specialist                                                                    │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Hello DeepLearningAI Team,                                                                                     │
│                                                                                                                 │
│  Thank you for reaching out to us regarding how to add memory capabilities to a Crew in CrewAI. I understand    │
│  the importance of leveraging memory-related functionalities to enhance the performance and intelligence of     │
│  your autonomous AI agents within your Crew. Below, I have provided a comprehensive guide on how to             │
│  effectively incorporate memory features into your Crew setup in CrewAI:                                        │
│                                                                                                                 │
│  1. Role-Based Agents: Begin by creating specialized agents within your Crew, each with defined roles,          │
│  expertise, and goals. Consider including memory-related roles such as researchers or analysts who require the  │
│  ability to store and retrieve information during task execution. These roles will enable your agents to        │
│  effectively manage information flow within the Crew.                                                           │
│                                                                                                                 │
│  2. Flexible Tools: Equip your agents with custom tools and APIs that allow them to interact with external      │
│  services and data sources. This will empower your agents to utilize memory-related functionalities, such as    │
│  storing and accessing information crucial for decision-making processes. Custom tools and APIs play a crucial  │
│  role in facilitating efficient memory management within your Crew.                                             │
│                                                                                                                 │
│  3. Intelligent Collaboration: Foster a collaborative environment where agents can work together seamlessly     │
│  within the Crew. Encourage sharing insights, coordinating tasks, and updating shared memory resources. This    │
│  collaboration ensures that memory resources are utilized effectively across agents, enhancing the overall      │
│  performance and intelligence of your Crew.                                                                     │
│                                                                                                                 │
│  4. Task Management: Define sequential or parallel workflows within your Crew to handle task dependencies       │
│  efficiently. Ensure that memory-related tasks are appropriately managed to maintain data integrity and         │
│  consistency. Proper task management is essential for optimizing memory usage within the Crew and achieving     │
│  desired task outcomes.                                                                                         │
│                                                                                                                 │
│  By incorporating these elements into your Crew setup in CrewAI, you can effectively add memory capabilities    │
│  to facilitate information storage, retrieval, and utilization within the collaborative AI environment. This    │
│  strategic approach will undoubtedly enhance the perfor

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 20dc2c77-b2ed-45b8-b994-5306d8929cf5                                                                     │
│  Agent: Support Quality Assurance Specialist                                                                    │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: e94d5468-9c5c-4380-b6fd-7914bd4898ee                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Hello DeepLearningAI Team,                                                                       │
│                                                                                                                 │
│  Thank you for reaching out to us regarding how to add memory capabilities to a Crew in CrewAI. I understand    │
│  the importance of leveraging memory-related functionalities to enhance the performance and intelligence of     │
│  your autonomous AI agents within your Crew. Below, I have provided a comprehensive guide on how to             │
│  effectively incorporate memory features into your Crew setup in CrewAI:                                        │
│                                                                                                                 │
│  1. Role-Based Agents: Begin by creating specialized agents within your Crew, each with defined roles,          │
│  expertise, and goals. Consider including memory-related roles such as researchers or analysts who require the  │
│  ability to store and retrieve information during task execution. These roles will enable your agents to        │
│  effectively manage information flow within the Crew.                                                           │
│                                                                                                                 │
│  2. Flexible Tools: Equip your agents with custom tools and APIs that allow them to interact with external      │
│  services and data sources. This will empower your agents to utilize memory-related functionalities, such as    │
│  storing and accessing information crucial for decision-making processes. Custom tools and APIs play a crucial  │
│  role in facilitating efficient memory management within your Crew.                                             │
│                                                                                                                 │
│  3. Intelligent Collaboration: Foster a collaborative environment where agents can work together seamlessly     │
│  within the Crew. Encourage sharing insights, coordinating tasks, and updating shared memory resources. This    │
│  collaboration ensures that memory resources are utilized effectively across agents, enhancing the overall      │
│  performance and intelligence of your Crew.                                                                     │
│                                                                                                                 │
│  4. Task Management: Define sequential or parallel workflows within your Crew to handle task dependencies       │
│  efficiently. Ensure that memory-related tasks are appropriately managed to maintain data integrity and         │
│  consistency. Proper task management is essential for optimizing memory usage within the Crew and achieving     │
│  desired task outcomes.                                                                                         │
│                                                                                                                 │
│  By incorporating these elements into your Crew setup in CrewAI, you can effectively add memory capabilities    │
│  to facilitate information storage, retrieval, and uti

In [39]:
from IPython.display import Markdown, display

clean = str(result).strip().removeprefix("```markdown").removesuffix("```").strip()
display(Markdown(clean))


Hello DeepLearningAI Team,

Thank you for reaching out to us regarding how to add memory capabilities to a Crew in CrewAI. I understand the importance of leveraging memory-related functionalities to enhance the performance and intelligence of your autonomous AI agents within your Crew. Below, I have provided a comprehensive guide on how to effectively incorporate memory features into your Crew setup in CrewAI:

1. Role-Based Agents: Begin by creating specialized agents within your Crew, each with defined roles, expertise, and goals. Consider including memory-related roles such as researchers or analysts who require the ability to store and retrieve information during task execution. These roles will enable your agents to effectively manage information flow within the Crew.

2. Flexible Tools: Equip your agents with custom tools and APIs that allow them to interact with external services and data sources. This will empower your agents to utilize memory-related functionalities, such as storing and accessing information crucial for decision-making processes. Custom tools and APIs play a crucial role in facilitating efficient memory management within your Crew.

3. Intelligent Collaboration: Foster a collaborative environment where agents can work together seamlessly within the Crew. Encourage sharing insights, coordinating tasks, and updating shared memory resources. This collaboration ensures that memory resources are utilized effectively across agents, enhancing the overall performance and intelligence of your Crew.

4. Task Management: Define sequential or parallel workflows within your Crew to handle task dependencies efficiently. Ensure that memory-related tasks are appropriately managed to maintain data integrity and consistency. Proper task management is essential for optimizing memory usage within the Crew and achieving desired task outcomes.

By incorporating these elements into your Crew setup in CrewAI, you can effectively add memory capabilities to facilitate information storage, retrieval, and utilization within the collaborative AI environment. This strategic approach will undoubtedly enhance the performance of your Crew in tackling complex tasks for DeepLearningAI.

If you require more detailed guidance on implementing memory features within your Crew using CrewAI, I recommend delving into the comprehensive documentation and resources provided by CrewAI. These valuable resources offer specific techniques and best practices to help you maximize the potential of memory functionalities within your Crew.

Should you have any further inquiries or require additional assistance, please do not hesitate to reach out. We are dedicated to supporting you in optimizing your Crew's performance and achieving success in your AI endeavors.

Best regards,

Senior Support Representative
CrewAI